In [ ]:
__author__ = "Alireza Sadabadi"
__copyright__ = "Copyright (c) 2024 Alireza Sadabadi. All rights reserved."
__credits__ = ["Alireza Sadabadi"]
__license__ = "Apache"
__version__ = "2.0"
__maintainer__ = "Alireza Sadabadi"
__email__ = "alirezasadabady@gmail.com"
__status__ = "Test"
__doc__ = "you can see the tutorials in https://youtube.com/@alirezasadabadi?si=d8o7LK_Ai1Hf68is"

In [470]:
import pandas as pd
import numpy as np
import ta.momentum

data = pd.read_csv("Candles.csv", index_col='time')

In [471]:
rsiLength=7; haRsiLength=15; upper=66; lower=28; risk=0.07; reward=0.14; atrPeriod=4; atrMultiplier=3
# مقادیر را ابتدا به مقیاس یک آر اس آی می بریم
# توجه شود که در این استراتژی دو مقدار مختلف برای دو آر اس آی داریم
data['closeRsi'] = ta.momentum.RSIIndicator(data["close"],window=haRsiLength).rsi()
data['openRsi'] = ta.momentum.RSIIndicator(data["open"],window=haRsiLength).rsi()
data['highRsi_Raw'] = ta.momentum.RSIIndicator(data["high"],window=haRsiLength).rsi()
data['lowRsi_Raw'] = ta.momentum.RSIIndicator(data["low"],window=haRsiLength).rsi()
data['highRsi'] = data[['highRsi_Raw','lowRsi_Raw']].max(axis=1)
data['lowRsi'] = data[['highRsi_Raw','lowRsi_Raw']].min(axis=1)

# محاسبه مقادیر هیکن آشی
data['hclose'] = (data['closeRsi']+data['openRsi']+data['highRsi']+data['lowRsi'])/4
data['hopen'] = data['openRsi']
data.dropna(inplace=True)

data.reset_index(inplace=True)
for i in range(1, len(data)):
    data.at[i,'hopen'] = (data.loc[i-1]['hopen']+data.loc[i-1]['hclose'])/2
data.set_index("time", inplace=True)
data['hhigh'] = data[['highRsi', 'hopen', 'hclose']].max(axis=1)
data['hlow'] = data[['lowRsi', 'hopen', 'hclose']].min(axis=1)

data['prehclose'] = data['hclose'].shift(1)
data['prehopen'] = data['hopen'].shift(1)

# محاسبه چندلر برای ورود قویتر به پوزیشن
data['atr'] = pd.Series(np.nan)
data['atr'] = data['high'] - data['low']
data['atr'] = data['atr'].rolling(window=atrPeriod).mean()

data['long'] = data['high'].rolling(window=atrPeriod).max() - (data['atr'] * atrMultiplier)
data['short'] = data['low'].rolling(window=atrPeriod).min() + (data['atr'] * atrMultiplier)

#buy
data['ce'] = np.where((data['close'] > data['short']) & (data['close'].shift(1) <= data['short'].shift(1)), 1, np.nan)
#sell
data['ce'] = np.where((data['close'] < data['long']) & (data['close'].shift(1) >= data['long'].shift(1)), -1, data.ce) 
data['ce'] = data.ce.ffill().fillna(0)

# محاسبه آر اس آی با طول دیگر به عنوان شاخص خروج از پوزیشن
data['rsi'] = ta.momentum.RSIIndicator(data["close"], window=rsiLength).rsi()
# محاسبه ای ام ای و ار اس آی برای شاخص ورود قوی تر به پوزیشن
data['ema'] = data["close"].ewm(min_periods=200,span=200,adjust=False).mean()
# data['rsiy'] = ta.momentum.RSIIndicator(data["close"], window=25).rsi()
# data['rsiw'] = ta.momentum.RSIIndicator(data["close"], window=100).rsi()
data.dropna(inplace=True)

        # (data.prehclose > lowerExtreme) & \
        # (data.rsiy > data.rsiw) & \
        # (data['enter_long']) == 1
buy = (data.prehclose < lower) & \
            (data.prehclose < data.prehopen) & \
            (data.hclose > data.hopen)

            # (data.prehclose < upperExtreme) & \
            # (data.rsiy < data.rsiw) & \
            # (data['enter_short'] == 1)
sell = (data.prehclose > upper) & \
            (data.prehclose > data.prehopen) & \
            (data.hclose < data.hopen)



data["position"]= np.where(buy, 1, np.nan)
data["position"]= np.where(sell, -1, data.position)

In [472]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9787 entries, 2025-01-27 07:25:00 to 2025-02-03 12:18:00
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   open         9787 non-null   float64
 1   high         9787 non-null   float64
 2   low          9787 non-null   float64
 3   close        9787 non-null   float64
 4   tick_volume  9787 non-null   int64  
 5   spread       9787 non-null   int64  
 6   real_volume  9787 non-null   int64  
 7   closeRsi     9787 non-null   float64
 8   openRsi      9787 non-null   float64
 9   highRsi_Raw  9787 non-null   float64
 10  lowRsi_Raw   9787 non-null   float64
 11  highRsi      9787 non-null   float64
 12  lowRsi       9787 non-null   float64
 13  hclose       9787 non-null   float64
 14  hopen        9787 non-null   float64
 15  hhigh        9787 non-null   float64
 16  hlow         9787 non-null   float64
 17  prehclose    9787 non-null   float64
 18  prehopen     9787 no

In [473]:
data = data.loc[:,["open", "high", "low", "close", "tick_volume", "position", "rsi"]]
data.reset_index(inplace=True)
data.columns = ['Local time', 'Open', 'High', 'Low', 'Close', 'Volume', 'signal', "rsi"]
data.index = pd.DatetimeIndex(data['Local time'])
data

,Local time,Open,High,Low,Close,Volume,signal,rsi
Local time,,,,,,,,
2025-01-27 07:25:00,2025-01-27 07:25:00,100372.77,100379.65,100272.13,100355.16,108,NaN,29.198286
2025-01-27 07:26:00,2025-01-27 07:26:00,100366.11,100366.11,100288.07,100351.90,100,NaN,28.880935
2025-01-27 07:27:00,2025-01-27 07:27:00,100338.44,100425.11,100332.35,100408.60,93,NaN,41.731659
2025-01-27 07:28:00,2025-01-27 07:28:00,100403.75,100517.52,100373.18,100495.87,85,NaN,56.006190
2025-01-27 07:29:00,2025-01-27 07:29:00,100494.12,100495.13,100382.29,100447.63,85,NaN,48.365175
...,...,...,...,...,...,...,...,...
2025-02-03 08:07:00,2025-02-03 08:07:00,94040.00,94047.02,93987.01,94013.92,84,NaN,55.848330
2025-02-03 08:08:00,2025-02-03 08:08:00,94015.92,94027.71,93883.44,93983.05,74,NaN,52.564531
2025-02-03 08:09:00,2025-02-03 08:09:00,93996.82,94035.50,93986.16,94015.65,47,NaN,55.768758


In [481]:
data[data['signal']==-1].count()

Local time    191
Open          191
High          191
Low           191
Close         191
Volume        191
signal        191
rsi           191
dtype: int64

In [475]:
def RiskReward(price, leverage=100, buy=True, risk=0.06, reward=0.12):        
    nb_decimal = str(price)[::-1].find(".")+2
    varDown = risk/leverage
    varUp = reward/leverage

    if buy:
        price_varDown = varDown * price
        price_varUp = varUp * price        
        tp = np.round(price + price_varUp, nb_decimal)
        sl = np.round(price - price_varDown, nb_decimal)            
    else:                
        price_varDown = varDown * price
        price_varUp = varUp * price        
        tp = np.round(price - price_varUp, nb_decimal)
        sl = np.round(price + price_varDown, nb_decimal)
    return tp, sl

In [476]:
def Signal():
    return data.signal

In [477]:
from backtesting import Strategy

class MyStrategy(Strategy):  
    def init(self):
        super().init()
        self.signal1 = self.I(Signal)

    def next(self):
        super().next() 
        upper=66; lower=28;
        if len(self.trades) > 0:
            if self.trades[-1].is_long and self.data.rsi[-1] > upper:
                self.trades[-1].close()                
            elif self.trades[-1].is_short and self.data.rsi[-1] < lower:
                self.trades[-1].close()
                        
        if self.signal1==1 and len(self.trades) == 0:
            tp, sl = RiskReward(self.data.Close[-1])
            self.buy(sl=sl, tp=tp, size=0.01)
        elif self.signal1==-1 and len(self.trades) == 0:
            tp, sl = RiskReward(price=self.data.Close[-1], buy=False)
            self.sell(sl=sl, tp=tp, size=0.01)

In [479]:
from backtesting import Backtest

backtest = Backtest(data, MyStrategy, cash=110000, commission=0.0)
result = backtest.run()
result

Start                     2025-01-27 07:25:00
End                       2025-02-03 12:18:00
Duration                      7 days 04:53:00
Exposure Time [%]                         0.0
Equity Final [$]                     110000.0
Equity Peak [$]                      110000.0
Return [%]                                0.0
Buy & Hold Return [%]                -5.13981
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
CAGR [%]                                  0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                   